In [ ]:
from Bio.PDB import PDBParser, PPBuilder
import nglview as nv

# Load structure
parser = PDBParser()
structure = parser.get_structure('Your_protein', 'test.pdb')

# Get chain A
chain_A = structure[0]['A']

# Extract sequence
ppb = PPBuilder()
sequence = ""
for pp in ppb.build_peptides(chain_A):
    sequence += str(pp.get_sequence())

print(sequence)

In [ ]:
# Show A Chain structure
view = nv.show_biopython(structure)
view

In [ ]:
# Show Whole structure
view = nv.show_biopython(chain_A)
view

In [ ]:
from Bio.PDB import *
import numpy as np

# Initialize parser
parser = PDBParser()

# Load structure from file
structure = parser.get_structure("Your_protein", "test.pdb")

# Get chain A
chain_A = structure[0]['A']

# Filter out only alpha carbon atoms
alpha_carbons = [atom for residue in chain_A for atom in residue if atom.get_id() == "CA"]

# Define function to calculate distance
def calculate_distance(atom1, atom2):
    diff_vector  = atom2.coord - atom1.coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

# Create distance matrix
num_atoms = len(alpha_carbons)
distance_matrix = np.zeros((num_atoms, num_atoms))

for i in range(num_atoms):
    for j in range(num_atoms):
        distance_matrix[i, j] = calculate_distance(alpha_carbons[i], alpha_carbons[j])

# Save distance matrix to a csv file
np.savetxt("distance_matrix.csv", distance_matrix, delimiter=",")


In [ ]:
distance_matrix.shape

In [ ]:
from Bio.PDB import PDBParser
import random

parser = PDBParser()
structure = parser.get_structure('Your_protein', 'test.pdb')

chain_A = structure[0]['A']

ppb = PPBuilder()

In [ ]:
# Using the method you mentioned to get the list of phi and psi angles
phi_psi_list = ppb.build_peptides(chain_A)[0].get_phi_psi_list()

# Separate the phi and psi angles into two lists
phi_angles = [angles[0] for angles in phi_psi_list if angles[0] is not None]
psi_angles = [angles[1] for angles in phi_psi_list if angles[1] is not None]
import math

phi_angles_deg = [math.degrees(angle) for angle in phi_angles]
psi_angles_deg = [math.degrees(angle) for angle in psi_angles]

phi_stdevs = [random.uniform(0, 5) for _ in range(len(phi_angles_deg))]
psi_stdevs = [random.uniform(0, 5) for _ in range(len(psi_angles_deg))]


In [ ]:
len(phi_angles_deg) , len(psi_angles_deg) , max(phi_angles_deg)

In [ ]:
from pyrosetta import *
from pyrosetta.rosetta.core.scoring.constraints import AtomPairConstraint
from pyrosetta.rosetta.core.scoring.func import HarmonicFunc
from pyrosetta.rosetta.core.id import AtomID
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.protocols.minimization_packing import MinMover


from pyrosetta.rosetta.core.scoring.constraints import DihedralConstraint
from pyrosetta.rosetta.core.scoring.func import CircularHarmonicFunc
from pyrosetta.rosetta.core.id import AtomID, TorsionID

# initialize PyRosetta
init()

# load your protein
pose = pose_from_sequence(sequence)

# Assume phi_angles and psi_angles are your precalculated lists of angles, which are of length N-1

# Set the phi angles for residues 2 to N
for i in range(2, pose.total_residue() + 1):
    pose.set_phi(i, phi_angles_deg[i-2])

# Set the psi angles for residues 1 to N-1
for i in range(1, pose.total_residue()):
    pose.set_psi(i, psi_angles_deg[i-1])

for i in range(len(distance_matrix)):
    for j in range(i+1, len(distance_matrix[i])):
        # create an AtomPairConstraint for the alpha carbons of residues i+1 and j+1
        atom1 = AtomID(2, i+1)  # 2 is the index for alpha carbon ('CA') in Rosetta atom numbering
        atom2 = AtomID(2, j+1)
        func = HarmonicFunc(distance_matrix[i][j], 0.1)  # Harmonic potential with standard deviation of 1.0
        constraint = AtomPairConstraint(atom1, atom2, func)

        # add the constraint to the pose
        pose.add_constraint(constraint)
        

for i in range(2, pose.total_residue() + 1):  # Rosetta uses 1-indexing
    # Create AtomID for the atoms forming the phi angle: C(i-1)-N(i)-CA(i)-C(i)
    atom1 = AtomID(4, i-1)  # C atom of previous residue
    atom2 = AtomID(1, i)  # N atom of current residue
    atom3 = AtomID(2, i)  # CA atom of current residue
    atom4 = AtomID(4, i)  # C atom of current residue
    
    # Create a CircularHarmonicFunc for the phi angle
    func = CircularHarmonicFunc(phi_angles_deg[i-2], phi_stdevs[i-2])

    # Create a DihedralConstraint for the phi angle
    constraint = DihedralConstraint(atom1, atom2, atom3, atom4, func )

    # Add the constraints to the pose
    pose.add_constraint(constraint)

In [ ]:
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.protocols.moves import MonteCarlo
from pyrosetta.rosetta.core.scoring import ScoreType

from pyrosetta.rosetta.core.kinematics import MoveMap


# Create a MoveMap that will allow backbone torsion angles to change
movemap = MoveMap()
movemap.set_bb(True)  # True means all backbone torsion angles are allowed to change


# Create a score function
scorefxn = get_score_function()  # Use the default score function

# Increase the weights of the constraint terms
scorefxn.set_weight(ScoreType.atom_pair_constraint, 10.0)
scorefxn.set_weight(ScoreType.dihedral_constraint, 10.0)



# Create a Monte Carlo mover
mc = MonteCarlo(pose, scorefxn, 1.0)  # The last parameter is the temperature

# Create a FastRelax mover
fast_relax = FastRelax(scorefxn, 1)  # The second parameter is the number of rounds
fast_relax.set_movemap(movemap)
# fast_relax.max_iter(100000000)  # Set maximum iterations
fast_relax.set_scorefxn(scorefxn)



In [11]:
from tqdm import tqdm

for _ in tqdm(range(10)):  # The number of iterations
    fast_relax.apply(pose)
    mc.boltzmann(pose)


In [ ]:

# # Create a MinMover for performing energy minimization
# min_mover = MinMover()
# min_mover.max_iter(100000000)  # Set maximum iterations to 1000

# min_mover.movemap(movemap)
# min_mover.score_function(get_score_function())  # Use the default score function

# # Perform energy minimization

# min_mover.apply(pose)


In [ ]:
pose.dump_pdb("output.pdb")
# Load structure
parser = PDBParser()
structure = parser.get_structure('Your_protein', 'output.pdb')
# Get chain A
chain_A_output = structure[0]['A']


In [ ]:
import nglview as nv
from ipywidgets import VBox

# Show chain_A_output structure
view1 = nv.show_biopython(chain_A_output)
view1
# Show chain_A structure
view2 = nv.show_biopython(chain_A)
view2
# Display side by side
VBox([view1, view2])
